In [17]:
### 수명 data 정리
import pandas as pd
import numpy as np

# Try opening with utf-8 encoding
try:
    df = pd.read_csv('life_avg.csv', encoding='utf-8')
except UnicodeDecodeError:
    # If error, try with cp949 encoding
    df = pd.read_csv('life_avg.csv', encoding='cp949')

#2010~2020년 data만
df = df.rename(columns={'지표연도': 'year', '시군구': 'City', '시도':'District', '평균 기대수명' : 'AVG'})
df = df[df['성별'] == '전체']
df = df[['City', 'year', 'District', 'AVG']]
df = df[df['City'] != '전체']
df = df[(df['year'] >= 2010) & (df['year'] <= 2020)]

print(df)

      City  year District    AVG
90     종로구  2010    서울특별시  82.25
93     종로구  2011    서울특별시  82.76
96     종로구  2012    서울특별시  82.93
99     종로구  2013    서울특별시  83.35
102    종로구  2014    서울특별시  83.66
...    ...   ...      ...    ...
9864   함양군  2020     경상남도  83.24
9867   거창군  2020     경상남도  83.22
9870   합천군  2020     경상남도  81.97
9876   제주시  2020  제주특별자치도  84.36
9879  서귀포시  2020  제주특별자치도  84.43

[2793 rows x 4 columns]


In [18]:
cities = ['부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '서울특별시','세종특별자치시']

for city in cities:
    # 광역시의 평균값 계산
    city_avg_df = df[df['District'] == city].groupby('year')['AVG'].mean().reset_index()

    # 해당 광역시 행 삭제
    df = df[df['District'] != city]

    # 광역시 평균값 행 추가
    for index, row in city_avg_df.iterrows():
        new_row = pd.DataFrame({'City':city, 'year':row['year'], 'District':city, 'AVG':row['AVG']}, index=[0])
        df = pd.concat([df, new_row], ignore_index=True)
df['year'] = df['year'].astype(int)
df['AVG'] = df['AVG'].round(2)

print(df)

         City  year District    AVG
0      수원시장안구  2010      경기도  81.54
1      수원시장안구  2011      경기도  81.93
2      수원시장안구  2012      경기도  82.00
3      수원시장안구  2013      경기도  82.13
4      수원시장안구  2014      경기도  82.43
...       ...   ...      ...    ...
2051  세종특별자치시  2016  세종특별자치시  82.83
2052  세종특별자치시  2017  세종특별자치시  83.27
2053  세종특별자치시  2018  세종특별자치시  83.44
2054  세종특별자치시  2019  세종특별자치시  83.86
2055  세종특별자치시  2020  세종특별자치시  84.38

[2056 rows x 4 columns]


In [19]:
# 광역시/특별시/특별자치시를 제외하고 뒤에서부터 '시'가 나올때까지 지움
excluded_cities = ['부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '서울특별시','세종특별자치시']

def remove_suffix(city):
    if city in excluded_cities:
        return city
    else:
        idx = city.rfind('시')
        if idx != -1:
            return city[:idx]
        else:
            return city

df['City'] = df['City'].apply(remove_suffix)

print(df)
print(df.dtypes)
df.to_csv('test.csv')

         City  year District    AVG
0          수원  2010      경기도  81.54
1          수원  2011      경기도  81.93
2          수원  2012      경기도  82.00
3          수원  2013      경기도  82.13
4          수원  2014      경기도  82.43
...       ...   ...      ...    ...
2051  세종특별자치시  2016  세종특별자치시  82.83
2052  세종특별자치시  2017  세종특별자치시  83.27
2053  세종특별자치시  2018  세종특별자치시  83.44
2054  세종특별자치시  2019  세종특별자치시  83.86
2055  세종특별자치시  2020  세종특별자치시  84.38

[2056 rows x 4 columns]
City         object
year          int32
District     object
AVG         float64
dtype: object


In [20]:
##pollution data랑 합치기
df_pollute = pd.read_csv('pollution_final.csv')
df_pollute['year'] = df_pollute['date'].astype(str).str[:4] # 문자열로 변환해서 year 추출
df_pollute.drop('date', axis=1, inplace=True)
df_pollute['year'] = df_pollute['year'].astype(int) # 다시 int로 바꿔서 합칠 준비

# city와 year이 일치하는 행들만 남기기
merged_df = df_pollute.merge(df[['City', 'year', 'AVG']], on=['City', 'year'], how='inner')

# avg에 결측치가 있는 행 제거하기
merged_df = merged_df.dropna(subset=['AVG'])

#column 재배치하고 index number 정리
merged_df = merged_df[['District', 'City', 'year', 'pm25', 'pm10', 'so2', 'o3', 'no2', 'co', 'AVG']]
merged_df = merged_df.reset_index(drop=True)

print(merged_df)
merged_df.to_csv('pollution_avg_final.csv')

      District City  year pm25  pm10    so2     o3    no2   co    AVG
0          강원도   강릉  2010  NaN  48.0  0.007  0.024   0.02  0.6  79.83
1          강원도   강릉  2010  NaN    51  0.006  0.022  0.022  0.6  79.83
2          강원도   강릉  2010  NaN  62.0  0.006  0.028   0.02  0.5  79.83
3          강원도   강릉  2010  NaN    51  0.006  0.036   0.02  0.5  79.83
4          강원도   강릉  2010  NaN    51  0.004  0.044  0.017  0.4  79.83
...        ...  ...   ...  ...   ...    ...    ...    ...  ...    ...
13867     충청북도   충주  2020   13    21  0.002  0.021  0.009  0.3  82.74
13868     충청북도   충주  2020   11    20  0.003  0.028  0.011  0.4  82.74
13869     충청북도   충주  2020   23    42  0.003  0.019  0.019  0.5  82.74
13870     충청북도   충주  2020   28    44  0.003  0.016  0.023  0.6  82.74
13871     충청북도   충주  2020  NaN    35  0.003  0.023  0.015  0.4  82.74

[13872 rows x 10 columns]
